# Artigo 7
* Aluno: Paulo Henrique Costa Gontijo
* Matrícula: 15/0143800
* Lição: **
* Notebooks: 
  * **
  * **

| Hitórico de versão | Data |
|:--------------------:|:------:|
| v1 | 24/01/2023 |

## Propósito
Este artigo tem a intenção de documentar o passo a passo da quarta lição do curso, mostrando os blocos de códigos, o resultado das execuções, eventuais problemas encontrados e, se houver, as possíveis soluções para os problemas.

## Tema
A lição 7 do curso FastAi contempla conceitos como Filtro Colaborativo, viés de usuário e produto. Dentre estes tópicos, são apresentadas técnicas de análise e treinamento de modelos específico para o tema.

In [1]:
# !pip install pyspark fastai fastbook -q

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# # from google.colab import files
# #
# # files.upload() #this will prompt you to upload the kaggle.json
#
# !ls -lha kaggle.json
#
# # Install kaggle API client
# !pip install -q kaggle
#
# # kaggle API client expects the file to be in ~/.kaggle
#
# # so move it there
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
#
# # we need to set permissions
# !chmod 600 /root/.kaggle/kaggle.json
#
# # check your directory before downloading the datasets
# !pwd
#
# # # list all available datasets
# # !kaggle datasets list
#
# # download the required dataset from kaggle
# !kaggle competitions download -c otto-recommender-system

# # If your file is a zip file you can unzip with the following code
# !unzip /content/otto-recommender-system.zip

In [4]:
# !unzip ./otto-recommender-system.zip

## Importação das Bibliotecas

In [5]:
import os
import pandas as pd

file_name = 'test.jsonl'
file_path = '/'.join([os.getcwd(), file_name])

## Criando Cluster Spark

In [6]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from os.path import abspath

warehouse_location = abspath('spark-warehouse')

    # init session
spark = SparkSession \
        .builder \
        .appName("otto-reccomender-competition-kaggle") \
        .config("spark.sql.warehouse.dir", abspath('spark-warehouse')) \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "true")\
        .config("spark.sql.shuffle.partitions", "1000")\
        .config("spark.driver.maxResultSize", "0")\
        .config("spark.driver.memory", "16g")\
        .config("spark.driver.memoryOverheadFactor", "0.20")\
        .config("spark.executor.pyspark.memory", "32g")\
        .config("spark.memory.offHeap.enabled", "true")\
        .config("spark.memory.offHeap.size", "8g")\
        .config("spark.checkpoint.compress", "true")\
        .enableHiveSupport() \
        .getOrCreate()

# show configured parameters
print(SparkConf().getAll())

# set log level
spark.sparkContext.setLogLevel("FATAL")

23/06/19 23:11:22 WARN Utils: Your hostname, gobellek-B660M-DS3H-DDR4 resolves to a loopback address: 127.0.1.1; using 192.168.0.37 instead (on interface enp3s0)
23/06/19 23:11:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/19 23:11:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[('spark.sql.shuffle.partitions', '1000'), ('spark.memory.offHeap.size', '8g'), ('spark.driver.memoryOverheadFactor', '0.20'), ('spark.executor.pyspark.memory', '32g'), ('spark.driver.memory', '16g'), ('spark.memory.offHeap.enabled', 'true'), ('spark.checkpoint.compress', 'true'), ('spark.app.name', 'otto-reccomender-competition-kaggle'), ('spark.app.submitTime', '1687227083064'), ('spark.sql.execution.arrow.pyspark.fallback.enabled', 'true'), ('spark.sql.catalogImplementation', 'hive'), ('spark.driver.maxResultSize', '0'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true'), ('spark.sql.warehouse.dir', '/home/gobellek/Documents/UnB/Tcc/spark-warehouse'), ('spark.sql.execution.arrow.pyspark.enabled', 'true')]


* Conferindo o instanciamento

In [7]:
spark

In [8]:
# import pyspark.pandas as ps
# ps.set_option('compute.ops_on_diff_frames', True)

* Leitura do arquivo disponibilizado na competição kaggle: [link.](https://www.kaggle.com/competitions/otto-recommender-system/overview/)

In [9]:
df_raw = spark.read \
    .format("json") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .json(file_path).persist()
df_raw.columns

['events', 'session']

* Conferindo número de partições e conteúdo

In [10]:
print(df_raw.rdd.getNumPartitions())

20


In [11]:
df_raw.show(10)

+--------------------+--------+
|              events| session|
+--------------------+--------+
|[{59625, 16617240...|12899779|
|[{1142000, 166172...|12899780|
|[{141736, 1661724...|12899781|
|[{1669402, 166172...|12899782|
|[{255297, 1661724...|12899783|
|[{1036375, 166172...|12899784|
|[{1784451, 166172...|12899785|
|[{955252, 1661724...|12899786|
|[{1682750, 166172...|12899787|
|[{245131, 1661724...|12899788|
+--------------------+--------+
only showing top 10 rows



* Contagem de linhas em arquivo bruto

In [12]:
df_raw.count()

1671803

In [13]:
df_raw.printSchema()

root
 |-- events: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- aid: long (nullable = true)
 |    |    |-- ts: long (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- session: long (nullable = true)



* Processamento de Json para formato tabular

In [14]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [15]:
def explode_df_json_to_tabular(df_raw):
    df_explode = df_raw.withColumn('events_explode', F.explode('events'))\
                    .withColumn('session', F.col('session').cast(T.IntegerType()))\
                    .withColumn('aid', F.col('events_explode.aid').cast(T.IntegerType()))\
                    .withColumn('ts', F.col('events_explode.ts'))\
                    .withColumn('type', F.col('events_explode.type'))\
                    .drop('events', 'events_explode')
    return df_explode

In [16]:
df_explode = explode_df_json_to_tabular(df_raw)
df_explode.show(10)

+--------+-------+-------------+------+
| session|    aid|           ts|  type|
+--------+-------+-------------+------+
|12899779|  59625|1661724000278|clicks|
|12899780|1142000|1661724000378|clicks|
|12899780| 582732|1661724058352|clicks|
|12899780| 973453|1661724109199|clicks|
|12899780| 736515|1661724136868|clicks|
|12899780|1142000|1661724155248|clicks|
|12899781| 141736|1661724000559|clicks|
|12899781| 199008|1661724022851|clicks|
|12899781|  57315|1661724170835|clicks|
|12899781| 194067|1661724246188|clicks|
+--------+-------+-------------+------+
only showing top 10 rows



* Contagem de linhas após o tratamento

In [17]:
df_explode.count()

6928123

* Schema pós tratamento

In [18]:
df_explode.printSchema()

root
 |-- session: integer (nullable = true)
 |-- aid: integer (nullable = true)
 |-- ts: long (nullable = true)
 |-- type: string (nullable = true)



* Categorização de coluna alvo string

In [19]:
from pyspark.ml.feature import StringIndexer

def transform_indexer(df_explode):
    indexer = StringIndexer(inputCol='type', outputCol='type_cat')
    indexer_fitted = indexer.fit(df_explode)
    df_indexed = indexer_fitted.transform(df_explode)
    df_indexed_type = df_indexed.withColumn('type_cat', F.col('type_cat').cast(T.IntegerType())).drop('type')
    df_indexed_type_renamed = df_indexed_type.withColumnRenamed('session','userCol')
    df_indexed_droped = df_indexed_type_renamed.dropDuplicates()
    df = df_indexed_droped.drop('ts')
    return df

In [20]:
df_indexed = transform_indexer(df_explode).persist()
df_indexed.show(10)

+--------+-------+--------+
| userCol|    aid|type_cat|
+--------+-------+--------+
|12900138| 633293|       1|
|12900140| 520771|       0|
|12900385|1370091|       0|
|12900801|1702775|       0|
|12901274|1764783|       0|
|12901436| 993764|       0|
|12901576|1484327|       0|
|12901644| 690731|       0|
|12901667|1092343|       0|
|12902016|1540384|       0|
+--------+-------+--------+
only showing top 10 rows



* Conferência de schema pós tratativas

In [21]:
df_indexed.printSchema()

root
 |-- userCol: integer (nullable = true)
 |-- aid: integer (nullable = true)
 |-- type_cat: integer (nullable = true)



## Análise de Clientes

* Quantidade de clientes únicos

In [22]:
df_indexed.select('userCol').distinct().count()

1671803

## Análise de Produtos

* Quantidade de produtos únicos

In [23]:
df_indexed.select('aid').distinct().count()

783486

# Escrita de DataFrame tratado

In [24]:
df_indexed.write.format('parquet')\
                       .mode("overwrite")\
                       .save('/'.join([os.getcwd(), 'refined-dataset']))

In [25]:
df_raw.unpersist()
df_explode.unpersist()
df_indexed.unpersist()

DataFrame[userCol: int, aid: int, type_cat: int]

# Lendo o dataset refinado

In [26]:
df_refined = spark.read.format('parquet').load('refined-dataset').persist()
df_refined.show()

+--------+-------+--------+
| userCol|    aid|type_cat|
+--------+-------+--------+
|12899845| 314210|       0|
|12899913|1531805|       0|
|12900133|1060926|       0|
|12900341| 142677|       0|
|12900389| 423932|       0|
|12901030| 348492|       0|
|12901439|  97836|       0|
|12901506|1144769|       0|
|12901684| 800913|       0|
|12902019|1559880|       0|
|12902234|  64218|       0|
|12902301| 235135|       0|
|12902383| 674590|       0|
|12902392|1336393|       0|
|12902834| 528280|       0|
|12903483|  83856|       2|
|12904044| 274237|       0|
|12904078| 190818|       0|
|12904260| 788288|       0|
|12904696| 527568|       1|
+--------+-------+--------+
only showing top 20 rows



In [27]:
# df_pre_moodel = df_refined.select(df_refined.columns).where(F.col('type_cat') == 2)
df_pre_moodel = df_refined
df_pre_moodel.show()

+--------+-------+--------+
| userCol|    aid|type_cat|
+--------+-------+--------+
|12899845| 314210|       0|
|12899913|1531805|       0|
|12900133|1060926|       0|
|12900341| 142677|       0|
|12900389| 423932|       0|
|12901030| 348492|       0|
|12901439|  97836|       0|
|12901506|1144769|       0|
|12901684| 800913|       0|
|12902019|1559880|       0|
|12902234|  64218|       0|
|12902301| 235135|       0|
|12902383| 674590|       0|
|12902392|1336393|       0|
|12902834| 528280|       0|
|12903483|  83856|       2|
|12904044| 274237|       0|
|12904078| 190818|       0|
|12904260| 788288|       0|
|12904696| 527568|       1|
+--------+-------+--------+
only showing top 20 rows



In [28]:
df_pre_moodel.select('userCol').distinct().count()

1671803

In [29]:
df_pre_moodel.count()

6924640

In [30]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

# (training, test) = df_pre_moodel.randomSplit([0.6, 0.4])
training = df_pre_moodel

In [31]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5,
          regParam=0.01,
          userCol="userCol",
          itemCol="aid",
          ratingCol="type_cat",
          coldStartStrategy="drop",
          implicitPrefs=True)
model = als.fit(training)

In [32]:
# # Evaluate the model by computing the RMSE on the test data
# predictions = model.transform(test)
# evaluator = RegressionEvaluator(metricName="rmse", labelCol="type_cat",
#                                 predictionCol="prediction")
# rmse = evaluator.evaluate(predictions)
# print("Root-mean-square error = " + str(rmse))

# Utilizando dataset de teste da competição

In [33]:
file_name = 'test.jsonl'
file_path = '/'.join([os.getcwd(), file_name])

In [34]:
df_test = spark.read \
    .format("json") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .json(file_path).persist()

In [35]:
df_test.show()

+--------------------+--------+
|              events| session|
+--------------------+--------+
|[{59625, 16617240...|12899779|
|[{1142000, 166172...|12899780|
|[{141736, 1661724...|12899781|
|[{1669402, 166172...|12899782|
|[{255297, 1661724...|12899783|
|[{1036375, 166172...|12899784|
|[{1784451, 166172...|12899785|
|[{955252, 1661724...|12899786|
|[{1682750, 166172...|12899787|
|[{245131, 1661724...|12899788|
|[{525156, 1661724...|12899789|
|[{1219653, 166172...|12899790|
|[{915175, 1661724...|12899791|
|[{1537160, 166172...|12899792|
|[{1181781, 166172...|12899793|
|[{1746099, 166172...|12899794|
|[{207754, 1661724...|12899795|
|[{4503, 166172400...|12899796|
|[{1335784, 166172...|12899797|
|[{379440, 1661724...|12899798|
+--------------------+--------+
only showing top 20 rows



In [36]:
df_test.count()

1671803

# Tratamento de subset para recomendações

In [37]:
df_pre_transform = explode_df_json_to_tabular(df_test)
df_pre_transform.count()

6928123

In [38]:
df_subset = transform_indexer(df_pre_transform).select('userCol').distinct()
df_subset.count()

1671803

In [39]:
df_pre_moodel.join(df_subset, df_pre_moodel.userCol == df_subset.userCol, 'inner')\
                    .drop(df_subset.userCol).select('userCol').distinct().count()

1671803

In [40]:
df_recommend = model.recommendForUserSubset(df_subset, 20)
df_recommend.printSchema()

root
 |-- userCol: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- aid: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



# Escrevendo Dataset de Recomendações em JSON

In [ ]:
df_recommend.write.format('json')\
                  .mode("overwrite")\
                  .save('/'.join([os.getcwd(), 'raw_predictions.json']))

In [49]:
df_recommend.unpersist()

DataFrame[userCol: int, recommendations: array<struct<aid:int,rating:float>>]

# Leitura de Dataset RAW_PREDICTIONS
Nesta etapa será realizada a conversão do objeto JSON em formato tabular, para se adequar ao formato de submissão da competição.

In [50]:
df_recommend_json = spark.read.format('json').load('./raw_predictions.json').persist()
df_recommend_json.printSchema()

root
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- aid: long (nullable = true)
 |    |    |-- rating: double (nullable = true)
 |-- userCol: long (nullable = true)



In [51]:
df_recommend_json.show(10, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|recommendations                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |userCol |
+---

* Tratamento JSON->Tabular

In [52]:
df_recommend_json_exploded = df_recommend_json.withColumn('recommendations_explode', F.explode('recommendations'))\
                                    .withColumn('aid', F.col('recommendations_explode.aid').cast(T.IntegerType()))\
                                    .drop('recommendations_explode', 'recommendations')
df_recommend_json_exploded.printSchema()

root
 |-- userCol: long (nullable = true)
 |-- aid: integer (nullable = true)



In [53]:
df_recommend_json_exploded.show(10)

+--------+-------+
| userCol|    aid|
+--------+-------+
|12899804|1848840|
|12899804|1848836|
|12899804|1848820|
|12899804|1848816|
|12899804|1848810|
|12899804|1848796|
|12899804|1848776|
|12899804|1848770|
|12899804|1848750|
|12899804|1848736|
+--------+-------+
only showing top 10 rows



# Agrupando top 20 itens por usuário

In [54]:
df_submission_raw = df_recommend_json_exploded.withColumn('userCol', F.col('userCol').cast(T.StringType()))\
                                                .groupby('userCol').agg(F.collect_list('aid').alias("labels")).persist()
df_submission_raw = df_submission_raw.withColumnRenamed('userCol', 'session_type')
df_submission_raw.show(10, truncate=False)

+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|session_type|labels                                                                                                                                                                              |
+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|12900014    |[485256, 714524, 1571865, 1495817, 1604220, 25964, 471073, 579690, 1236775, 1257293, 832192, 1043508, 272744, 1626731, 480314, 345875, 1829291, 1022566, 508883, 615566]            |
|12903226    |[1848840, 1848836, 1848820, 1848816, 1848810, 1848796, 1848776, 1848770, 1848750, 1848736, 1848720, 1848710, 1848706, 1848676, 1848660, 1848656, 1848650, 1848640, 1848626, 1848616]|
|12903765    |[18488

In [55]:
df_submission_raw.count()

1671803

# Criando dataset no formato de submissão

In [56]:
users_list_raw = [r[0] for r in df_submission_raw.select('session_type').toLocalIterator()]
users_list_raw[:5]

['12900014', '12903226', '12903765', '12904744', '12905817']

In [57]:
users_list_clicks = [s + '_clicks' for s in users_list_raw]
users_list_clicks[:5]

['12900014_clicks',
 '12903226_clicks',
 '12903765_clicks',
 '12904744_clicks',
 '12905817_clicks']

In [58]:
users_list_carts = [s + '_carts' for s in users_list_raw]
users_list_carts[:5]

['12900014_carts',
 '12903226_carts',
 '12903765_carts',
 '12904744_carts',
 '12905817_carts']

In [59]:
users_list_orders = [s + '_orders' for s in users_list_raw]
users_list_orders[:5]

['12900014_orders',
 '12903226_orders',
 '12903765_orders',
 '12904744_orders',
 '12905817_orders']

In [60]:
labels_list = [r[0] for r in df_submission_raw.select('labels').toLocalIterator()]
labels_list[:5]

[[485256,
  714524,
  1571865,
  1495817,
  1604220,
  25964,
  471073,
  579690,
  1236775,
  1257293,
  832192,
  1043508,
  272744,
  1626731,
  480314,
  345875,
  1829291,
  1022566,
  508883,
  615566],
 [1848840,
  1848836,
  1848820,
  1848816,
  1848810,
  1848796,
  1848776,
  1848770,
  1848750,
  1848736,
  1848720,
  1848710,
  1848706,
  1848676,
  1848660,
  1848656,
  1848650,
  1848640,
  1848626,
  1848616],
 [1848840,
  1848836,
  1848820,
  1848816,
  1848810,
  1848796,
  1848776,
  1848770,
  1848750,
  1848736,
  1848720,
  1848710,
  1848706,
  1848676,
  1848660,
  1848656,
  1848650,
  1848640,
  1848626,
  1848616],
 [1848840,
  1848836,
  1848820,
  1848816,
  1848810,
  1848796,
  1848776,
  1848770,
  1848750,
  1848736,
  1848720,
  1848710,
  1848706,
  1848676,
  1848660,
  1848656,
  1848650,
  1848640,
  1848626,
  1848616],
 [1848840,
  1848836,
  1848820,
  1848816,
  1848810,
  1848796,
  1848776,
  1848770,
  1848750,
  1848736,
  1848720,
  18487

In [61]:
df_clicks = spark.createDataFrame(zip(users_list_clicks, labels_list), ['session_type', 'labels'])
df_clicks.show(5)

+---------------+--------------------+
|   session_type|              labels|
+---------------+--------------------+
|12900014_clicks|[485256, 714524, ...|
|12903226_clicks|[1848840, 1848836...|
|12903765_clicks|[1848840, 1848836...|
|12904744_clicks|[1848840, 1848836...|
|12905817_clicks|[1848840, 1848836...|
+---------------+--------------------+
only showing top 5 rows



Current mem limits: -1 of max -1

Setting mem limits to 34359738368 of max 34359738368



In [62]:
df_carts = spark.createDataFrame(zip(users_list_carts, labels_list), ['session_type', 'labels'])
df_carts.show(5)

+--------------+--------------------+
|  session_type|              labels|
+--------------+--------------------+
|12900014_carts|[485256, 714524, ...|
|12903226_carts|[1848840, 1848836...|
|12903765_carts|[1848840, 1848836...|
|12904744_carts|[1848840, 1848836...|
|12905817_carts|[1848840, 1848836...|
+--------------+--------------------+
only showing top 5 rows



Current mem limits: -1 of max -1

Setting mem limits to 34359738368 of max 34359738368



In [63]:
df_orders = spark.createDataFrame(zip(users_list_orders, labels_list), ['session_type', 'labels'])
df_orders.show(5)

+---------------+--------------------+
|   session_type|              labels|
+---------------+--------------------+
|12900014_orders|[485256, 714524, ...|
|12903226_orders|[1848840, 1848836...|
|12903765_orders|[1848840, 1848836...|
|12904744_orders|[1848840, 1848836...|
|12905817_orders|[1848840, 1848836...|
+---------------+--------------------+
only showing top 5 rows



Current mem limits: -1 of max -1

Setting mem limits to 34359738368 of max 34359738368



In [64]:
from functools import reduce
from pyspark.sql import DataFrame

dfs = [df_clicks,df_carts,df_orders]
df_submission = reduce(DataFrame.unionAll, dfs).persist()
df_submission = df_submission.withColumn("labels",F.concat_ws(" ",F.col("labels")))
df_submission.show()

Current mem limits: -1 of max -1

Setting mem limits to 34359738368 of max 34359738368



+---------------+--------------------+
|   session_type|              labels|
+---------------+--------------------+
|12900014_clicks|485256 714524 157...|
|12903226_clicks|1848840 1848836 1...|
|12903765_clicks|1848840 1848836 1...|
|12904744_clicks|1848840 1848836 1...|
|12905817_clicks|1848840 1848836 1...|
|12906542_clicks|1848840 1848836 1...|
|12906954_clicks|1848840 1848836 1...|
|12908578_clicks|1848840 1848836 1...|
|12908814_clicks|1848840 1848836 1...|
|12909880_clicks|1848840 1848836 1...|
|12910609_clicks|1848840 1848836 1...|
|12910767_clicks|1848840 1848836 1...|
|12914402_clicks|1043508 508883 29...|
|12916785_clicks|1848840 1848836 1...|
|12918158_clicks|1848840 1848836 1...|
|12918726_clicks|1848840 1848836 1...|
|12920450_clicks|1848840 1848836 1...|
|12923982_clicks|1848840 1848836 1...|
|12925451_clicks|1848840 1848836 1...|
|12929071_clicks|1848840 1848836 1...|
+---------------+--------------------+
only showing top 20 rows



In [65]:
df_submission.count()

Current mem limits: 34359738368 of max 34359738368

Current mem limits: -1 of max -1

Setting mem limits to 34359738368 of max 34359738368

Current mem limits: -1 of max -1

Setting mem limits to 34359738368 of max 34359738368

Current mem limits: -1 of max -1

Setting mem limits to 34359738368 of max 34359738368

Current mem limits: -1 of max -1

Setting mem limits to 34359738368 of max 34359738368

Current mem limits: -1 of max -1

Setting mem limits to 34359738368 of max 34359738368

Current mem limits: -1 of max -1

Setting mem limits to 34359738368 of max 34359738368

Current mem limits: -1 of max -1

Setting mem limits to 34359738368 of max 34359738368

Current mem limits: -1 of max -1

Setting mem limits to 34359738368 of max 34359738368

Current mem limits: -1 of max -1

Setting mem limits to 34359738368 of max 34359738368

Current mem limits: -1 of max -1

Setting mem limits to 34359738368 of max 34359738368

Current mem limits: -1 of max -1

Setting mem limits to 34359738368 

5015409

# Escrevendo dataset de submissão

In [66]:
df_submission.orderBy('session_type').coalesce(1)\
                                     .write.format('csv')\
                                     .mode("overwrite")\
                                     .option("header", True)\
                                     .save('/'.join([os.getcwd(), 'submission']))

# Referências Bibliográficas

> https://github.com/Kaggle/kaggle-api, acessado em 24/01/2023.

> https://www.youtube.com/watch?v=aBNQzWV_UmE, acessado em 24/01/2023.

> https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/968100988546031/157591980591166/8836542754149149/latest.html, acessado em 25/01/2023.

> https://blog.clairvoyantsoft.com/apache-spark-out-of-memory-issue-b63c7987fff, acessado em 25/01/2023.

> https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_ps.html, acessado em 26/01/2023.

> https://docs.fast.ai/collab.html, acessado em 24/01/2023.

> https://course.fast.ai/Lessons/lesson7.html, acessado em 24/01/2023.

> https://github.com/fastai/fastbook/blob/master/08_collab.ipynb, acessado em 24/01/2023.

> https://www.kaggle.com/code/vchulski/tutorial-collaborative-filtering-with-pyspark

> https://medium.com/analytics-vidhya/crafting-recommendation-engine-in-pyspark-a7ca242ad40a

> https://www.freecodecamp.org/news/8-clustering-algorithms-in-machine-learning-that-all-data-scientists-should-know/

> https://www.youtube.com/watch?v=fpqa0_U4zb4

> https://www.youtube.com/watch?v=58OjaDH2FI0&t=742s

> https://spark.apache.org/docs/latest/ml-collaborative-filtering.html

> https://spark.apache.org/docs/latest/configuration.html#memory-management